_This project is a guided project from DataQuest. With complete honesty, I put my best effort in formulating the code for both the data cleaning and analysis sections before referring to the provided ["solution"](https://github.com/dataquestio/solutions/blob/master/Mission356Solutions.ipynb). My goal in completing this project is to improve my skills in data science/analysis. This project was completed on July 9, 2020. &mdash; Zeth De Luna_

# Exploring Hacker News Posts

In this project, we will be working with a data set of submissions to popular technology site [Hacker News](https://news.ycombinator.com/). Hacker News is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

Below are descriptions of the columns:

* `id`: The unique identifier from Hacker News for the post
* `title`: The title of the post
* `url`: The URL that the posts links to, if the post has a URL
* `num_points`: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
* `num_comments`: The number of comments that were made on the post
* `author`: The username of the person who submitted the post
* `created_at`: The date and time at which the post was submitted

Here are the column names of the data set:

In [1]:
from csv import reader

opened_file = open('HN_posts_year_to_Sep_26_2016.csv') # opens file
read_file = reader(opened_file) # reads file
list_HN = list(read_file) # converst file into a list of lists
HN_header = list_HN[0] # data set header
HN = list_HN[1:] # data set

print(HN_header)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In this project, we're specifically interested in posts whose titles begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question. Below are some examples:

`Ask HN: How to improve my personal website?
Ask HN: Am I the only one outraged by Twitter shutting down share counts?
Ask HN: Aby recent changes to CSS that broke mobile?`

Similarly, users submit `Show HN` posts to show the Hacker News community a project, product, or just generally something interesting. Below are some examples:

`Show HN: Wio Link ESP 8266 Based Web of Things Hardware Development Platform'
Show HN: Something pointless I made
Show HN: Shanhu.io, a programming playground powered by e8vm`

We'll compare these two types of posts to determine the following:

* Do `Ask HN` or `Show HN` receive more comments on average?
* Do posts created at a certain time receive more comments on average?

## Extracting Ask HN and Show HN Posts That Have Received Comments

Based on the goal of this project, it would make sense to remove all the posts where `num_comments` is `0`. Since we're trying to find the posts with the most comments, by first cleaning out the posts with 0 comments we could make things a little easier.

In [2]:
HN_comments = [] # empty list that will contain posts with comments

for post in HN: # iterates through each row in the data set
    num_comments = post[4] # number of comments is in index 4 for each row
    if num_comments is not '0': # appends posts to HN_comments only if the post has comments
        HN_comments.append(post)
        
print('HN before: {} posts'.format(len(HN)))
print('HN without 0-comment posts: {} posts'.format(len(HN_comments)))

HN before: 293119 posts
HN without 0-comment posts: 80401 posts


We can see that the size of our data set was reduced significantly just be removing posts that did not receive any comments. 

Now, to make viewing the data set a little bit easier, we'll create a function called `display_dataset()` that prints out each row of the data set, separated by an empty line.

In [3]:
def display_dataset(dataset, start, end):
    data = dataset[start:end]
    for row in data:
        print(row, '\n')

In [4]:
display_dataset(HN_comments, 0, 5)

['12578975', 'Saving the Hassle of Shopping', 'https://blog.menswr.com/2016/09/07/whats-new-with-your-style-feed/', '1', '1', 'bdoux', '9/26/2016 3:13'] 

['12578908', 'Ask HN: What TLD do you use for local development?', '', '4', '7', 'Sevrene', '9/26/2016 2:53'] 

['12578822', 'Amazons Algorithms Dont Find You the Best Deals', 'https://www.technologyreview.com/s/602442/amazons-algorithms-dont-find-you-the-best-deals/', '1', '1', 'yarapavan', '9/26/2016 2:26'] 

['12578694', 'Emergency dose of epinephrine that does not cost an arm and a leg', 'http://m.imgur.com/gallery/th6Ua', '2', '1', 'dredmorbius', '9/26/2016 1:54'] 

['12578624', 'Phone Makers Could Cut Off Drivers. So Why Dont They?', 'http://www.nytimes.com/2016/09/25/technology/phone-makers-could-cut-off-drivers-so-why-dont-they.html', '4', '1', 'danso', '9/26/2016 1:37'] 



Recall that our goal has only to do with posts with titles that begin with either `Ask HN` or `Show HN`. So, we'll remove all other posts. To do this, we can use the string method `startswith`, which inspects a string based on the input parameter. For example, we can check if `post1` starts with `Ask HN` by implementing `post1.startswith('Ask HN')`. Some posts might actually be `Ask HN` posts, but are typed as `ask hn`, so they wouldn't be counted. To account for that, we can use the string method `lower`, which returns a lowercase version of the starting string.

Let's begin by separating the posts into three lists: `ask_posts`, `show_posts`, and `other_posts`.

In [5]:
ask_posts = []
show_posts = []
other_posts = []

for post in HN_comments:
    title = post[1] # title in HN_dataset is in index 1
    title = title.lower() # converts title string to all lowercase
    if title.startswith('ask hn'):
        ask_posts.append(post)
    elif title.startswith('show hn'):
        show_posts.append(post)
    else:
        other_posts.append(post)
        
# check the number of posts in each list
print('ask_posts has {} posts'.format(len(ask_posts)))
print('show_posts has {} posts'.format(len(show_posts)))
print('other_posts has {} posts'.format(len(other_posts)))

ask_posts has 6911 posts
show_posts has 5059 posts
other_posts has 68431 posts


To quickly check if the lists are correct, we'll take a look at the first few rows of the `ask_posts` and `show_posts` lists.

In [6]:
print('ask_posts:')
display_dataset(ask_posts, 0, 4)
print('show_posts:')
display_dataset(show_posts, 0, 4)

ask_posts:
['12578908', 'Ask HN: What TLD do you use for local development?', '', '4', '7', 'Sevrene', '9/26/2016 2:53'] 

['12578522', 'Ask HN: How do you pass on your work when you die?', '', '6', '3', 'PascLeRasc', '9/26/2016 1:17'] 

['12577870', 'Ask HN: Why join a fund when you can be an angel?', '', '1', '3', 'anthony_james', '9/25/2016 22:48'] 

['12577647', 'Ask HN: Someone uses stock trading as passive income?', '', '5', '2', '00taffe', '9/25/2016 21:50'] 

show_posts:
['12577142', 'Show HN: Jumble  Essays on the go #PaulInYourPocket', 'https://itunes.apple.com/us/app/jumble-find-startup-essay/id1150939197?ls=1&mt=8', '1', '1', 'ryderj', '9/25/2016 20:06'] 

['12576813', 'Show HN: Learn Japanese Vocab via multiple choice questions', 'http://japanese.vul.io/', '1', '1', 'soulchild37', '9/25/2016 19:06'] 

['12576090', 'Show HN: Markov chain Twitter bot. Trained on comments left on Pornhub', 'https://twitter.com/botsonasty', '3', '1', 'keepingscore', '9/25/2016 16:50'] 

['1257

Above, we can see that out of the about 80,000 posts, there are only about 7,000 ask posts and 5,000 show posts. 

## Calculating Average Number of Comments for Ask Posts and Show Posts

Next, let's determine if ask posts or show posts receive more comments on average.

In [7]:
# find the average number of comments in ask posts
total_ask_comments = 0

for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments
    
total_ask_posts = len(ask_posts) 
avg_ask_comments = total_ask_comments / total_ask_posts

# find the average number of comments in show posts
total_show_comments = 0

for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments
    
total_show_posts = len(show_posts)
avg_show_comments = total_show_comments / total_show_posts

print('Average Number of Comments in Ask Posts: {}'.format(avg_ask_comments))
print('Average Number of Comments in Show Posts: {}'.format(avg_show_comments))

Average Number of Comments in Ask Posts: 13.744175951381855
Average Number of Comments in Show Posts: 9.810832180272781


From our calculations above, we can see that, on average, ask posts (13.74 comments per post) receive more comments than show posts (9.81 comments per post).

## Evaluating Number of Comments in Ask Posts Based on Time of Posting

Here, we'll determine if ask posts created at a certain _time_ are more likely to attract more comments. To do this, we will:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

Let's begin the first step.

In [8]:
from datetime import datetime as dt

time_comments = []

for post in ask_posts:
    created_at = post[6]
    num_comments = post[4]
    tc_list = [created_at, int(num_comments)]
    time_comments.append(tc_list)
    
counts_by_hour = {}
comments_by_hour = {}

for row in time_comments:
    dayt = dt.strptime(row[0], "%m/%d/%Y %H:%M")
    hour = dayt.strftime("%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]

We've created two dictionaries:

* `counts_by_hour` contains the number of ask posts created during each hour of the day
* `comments_by_hour` contains the number of comments on ask posts during each corresponding hour

We're going to use these dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [9]:
avg_comm_per_hour = []

for hour in counts_by_hour:
    comments = int(comments_by_hour[hour])
    posts = int(counts_by_hour[hour])
    avg = comments / posts
    avg_comm_per_hour.append([hour, avg])

In [10]:
display_dataset(avg_comm_per_hour, 0, -1)

['02', 13.198237885462555] 

['01', 9.367713004484305] 

['22', 11.749128919860627] 

['21', 11.056511056511056] 

['19', 9.414285714285715] 

['17', 13.73019801980198] 

['15', 39.66809421841542] 

['14', 13.153439153439153] 

['13', 22.2239263803681] 

['11', 11.143426294820717] 

['10', 13.757990867579908] 

['09', 8.392045454545455] 

['07', 10.095541401273886] 

['03', 10.160377358490566] 

['16', 10.76144578313253] 

['08', 12.43157894736842] 

['00', 9.857142857142858] 

['23', 8.322463768115941] 

['20', 11.38265306122449] 

['18', 10.789823008849558] 

['12', 15.452554744525548] 

['04', 12.688172043010752] 

['06', 9.017045454545455] 



Above, we acheived a list of lists containing the average number of comments per post for each hour. To make it easier to find the highest average number of comments per post per hour, we'll sort the lists by descending average number of comments.

In [11]:
# switching the list format to [average, hour]

avg_comm_swap = []

for row in avg_comm_per_hour:
    hour = row[0]
    avg = row[1]
    avg_comm_swap.append([avg, hour])
    
print(avg_comm_swap)

[[13.198237885462555, '02'], [9.367713004484305, '01'], [11.749128919860627, '22'], [11.056511056511056, '21'], [9.414285714285715, '19'], [13.73019801980198, '17'], [39.66809421841542, '15'], [13.153439153439153, '14'], [22.2239263803681, '13'], [11.143426294820717, '11'], [13.757990867579908, '10'], [8.392045454545455, '09'], [10.095541401273886, '07'], [10.160377358490566, '03'], [10.76144578313253, '16'], [12.43157894736842, '08'], [9.857142857142858, '00'], [8.322463768115941, '23'], [11.38265306122449, '20'], [10.789823008849558, '18'], [15.452554744525548, '12'], [12.688172043010752, '04'], [9.017045454545455, '06'], [11.139393939393939, '05']]


In [14]:
# sorting the lists by hour descending

avg_comm_sorted = sorted(avg_comm_swap, reverse=True)

Now that our list is nicely sorted, we'll display the top 5 hours with the most comments.

In [19]:
print("Top 5 Hours for Ask Posts Comments", '\n')

for row in avg_comm_sorted[:5]:
    dt_hour = dt.strptime(row[1], "%H")
    hour = dt.strftime(dt_hour, "%H:%M")
    avg = row[0]
    print("{}: {:.2f} average comments per post".format(hour, avg))

Top 5 Hours for Ask Posts Comments 

15:00: 39.67 average comments per post
13:00: 22.22 average comments per post
12:00: 15.45 average comments per post
10:00: 13.76 average comments per post
17:00: 13.73 average comments per post


First, if we take a look at the data set's [description](https://www.kaggle.com/hacker-news/hacker-news-posts) we will find that the times were recorded in the Eastern Standard Time zone in the US (EST). As we can see from the data, the time with the highest average comments per post is at 3:00PM EST with an average of 39.67 comments per post. This is followed by 1:00PM EST with an average of 22.22 comments per post, and 12:00PM EST with an average of 15.45 comments per post.

__Conclusions:__ In the introduction, we discussed how Hacker News is a popular technology site where user-submitted posts are voted and commented on and is extremely popular amongst start-ups. So, the more votes and comments that a start-up receives, the higher exposure that the start-up has to potential supporters, investors, or consumers. Based on the data and our analysis, if your goal is to get the most exposure on your post, the best time to submit a post is at 3:00PM EST.